# pandas

# Groupby
- SQL groupby명령어와같음
- split, apply, combine
- 과정을거쳐연산함

In [3]:
import pandas as pd
# data
data = {'Team' : ['riders', 'riders', 'devils', 'devils', 'kings', 'kings', 'kings', 'kings', 'riders' ,'royals', 'royals', 'riders'],
        'Rank' : [1, 2, 2, 3, 3, 4, 1, 1, 2, 4, 1, 2],
        'Year' : [2014, 2015, 2014, 2015, 2014, 2015, 2016, 2017, 2016, 2014, 2015, 2017],
        'Points' : [876, 789, 863, 673, 741, 812, 756, 788, 694, 701, 804, 690]}

df = pd.DataFrame(data, columns = ['Team', 'Rank', 'Year', 'Points'])
df

,Team,Rank,Year,Points
0,riders,1,2014,876
1,riders,2,2015,789
2,devils,2,2014,863
3,devils,3,2015,673
4,kings,3,2014,741
5,kings,4,2015,812
6,kings,1,2016,756
7,kings,1,2017,788
8,riders,2,2016,694
9,royals,4,2014,701


In [4]:
# groupby
df.groupby('Team')['Points'].mean()

Team
devils    768.00
kings     774.25
riders    762.25
royals    752.50
Name: Points, dtype: float64

In [5]:
# 한개 이상의 column을 묶을 수 있음
df.groupby(['Team', 'Year'])['Points'].mean()

Team    Year
devils  2014    863
        2015    673
kings   2014    741
        2015    812
        2016    756
        2017    788
riders  2014    876
        2015    789
        2016    694
        2017    690
royals  2014    701
        2015    804
Name: Points, dtype: int64

# Hierarchical index
- Groupby 명령의 결과물도 결국은 dataframe
- 두 개의 column으로 groupby를 할 경우, index가 두개 생성

In [9]:
h_index = df.groupby(['Team', 'Year'])['Points'].sum()
h_index.index

MultiIndex([('devils', 2014),
            ('devils', 2015),
            ( 'kings', 2014),
            ( 'kings', 2015),
            ( 'kings', 2016),
            ( 'kings', 2017),
            ('riders', 2014),
            ('riders', 2015),
            ('riders', 2016),
            ('riders', 2017),
            ('royals', 2014),
            ('royals', 2015)],
           names=['Team', 'Year'])

In [10]:
# unstack() - Group으로 묶여진 데이터를 matrix 형태로 전환해줌
h_index.unstack()

Year,2014,2015,2016,2017
Team,,,,
devils,863.0,673.0,NaN,NaN
kings,741.0,812.0,756.0,788.0
riders,876.0,789.0,694.0,690.0
royals,701.0,804.0,NaN,NaN


In [11]:
# Hierarchical index – swaplevel - Index level을 변경할 수 있음
h_index.swaplevel()

Year  Team  
2014  devils    863
2015  devils    673
2014  kings     741
2015  kings     812
2016  kings     756
2017  kings     788
2014  riders    876
2015  riders    789
2016  riders    694
2017  riders    690
2014  royals    701
2015  royals    804
Name: Points, dtype: int64

In [13]:
h_index.swaplevel().sort_index()

Year  Team  
2014  devils    863
      kings     741
      riders    876
      royals    701
2015  devils    673
      kings     812
      riders    789
      royals    804
2016  kings     756
      riders    694
2017  kings     788
      riders    690
Name: Points, dtype: int64

In [14]:
# Hierarchical index – operations - Index level을 기준으로 기본 연산 수행 가능
h_index.sum(level = 0)

Team
devils    1536
kings     3097
riders    3049
royals    1505
Name: Points, dtype: int64

In [15]:
h_index.sum(level = 1)

Year
2014    3181
2015    3078
2016    1450
2017    1478
Name: Points, dtype: int64

In [16]:
# grouped - groupby에 의해 split된 상태를 추출 가능
grouped = df.groupby('Team')

for name, group in grouped :    # Tuple 형태로 그룹의 key값 value값이 추출됨
    print(name)
    print(group)

devils
     Team  Rank  Year  Points
2  devils     2  2014     863
3  devils     3  2015     673
kings
    Team  Rank  Year  Points
4  kings     3  2014     741
5  kings     4  2015     812
6  kings     1  2016     756
7  kings     1  2017     788
riders
      Team  Rank  Year  Points
0   riders     1  2014     876
1   riders     2  2015     789
8   riders     2  2016     694
11  riders     2  2017     690
royals
      Team  Rank  Year  Points
9   royals     4  2014     701
10  royals     1  2015     804


In [17]:
# 특정 key 값을 가진 그룹의 정보만 추출 가능
grouped.get_group('devils')

,Team,Rank,Year,Points
2,devils,2,2014,863
3,devils,3,2015,673


## grouped
- 추출된 group 정보에는 세 가지 유형의 apply가 가능함
- Aggregation: 요약된 통계정보를 추출해 줌
- Transformation: 해당 정보를 변환해줌
- Filtration: 특정 정보를 제거 하여 보여주는 필터링 기능

In [18]:
# aggregation
grouped.agg(sum)

,Rank,Year,Points
Team,,,
devils,5,4029,1536
kings,9,8062,3097
riders,7,8062,3049
royals,5,4029,1505


In [19]:
import numpy as np
grouped.agg(np.mean)

,Rank,Year,Points
Team,,,
devils,2.50,2014.5,768.00
kings,2.25,2015.5,774.25
riders,1.75,2015.5,762.25
royals,2.50,2014.5,752.50


In [20]:
# 특정 컬럼에 여러개의 function을 Apply 할 수도 있음
grouped['Points'].agg([np.sum, np.mean, np.std])

,sum,mean,std
Team,,,
devils,1536,768.00,134.350288
kings,3097,774.25,31.899582
riders,3049,762.25,88.567771
royals,1505,752.50,72.831998


In [22]:
# transformation - Aggregation과 달리 key 값 별로 요약된 정보가 아님, 개별 데이터의 변환을 지원함
display(df)
score = lambda x : (x)
grouped.transform(score)

,Team,Rank,Year,Points
0,riders,1,2014,876
1,riders,2,2015,789
2,devils,2,2014,863
3,devils,3,2015,673
4,kings,3,2014,741
5,kings,4,2015,812
6,kings,1,2016,756
7,kings,1,2017,788
8,riders,2,2016,694
9,royals,4,2014,701


,Rank,Year,Points
0,1,2014,876
1,2,2015,789
2,2,2014,863
3,3,2015,673
4,3,2014,741
5,4,2015,812
6,1,2016,756
7,1,2017,788
8,2,2016,694
9,4,2014,701


In [24]:
score = lambda x : (x.max())
grouped.transform(score)    # 단 max나 min처럼 Series 데이터에 적용되는 데이터들은
                            # Key 값을 기준으로 Grouped 된 데이터 기준

,Rank,Year,Points
0,2,2017,876
1,2,2017,876
2,3,2015,863
3,3,2015,863
4,4,2017,812
5,4,2017,812
6,4,2017,812
7,4,2017,812
8,2,2017,876
9,4,2015,804


In [25]:
score = lambda x : (x - x.mean()) / x.std()
grouped.transform(score)

,Rank,Year,Points
0,-1.500000,-1.161895,1.284327
1,0.500000,-0.387298,0.302029
2,-0.707107,-0.707107,0.707107
3,0.707107,0.707107,-0.707107
4,0.500000,-1.161895,-1.042333
5,1.166667,-0.387298,1.183401
6,-0.833333,0.387298,-0.572108
7,-0.833333,1.161895,0.431040
8,0.500000,0.387298,-0.770596
9,0.707107,-0.707107,-0.707107


In [27]:
# filter - 특정 조건으로 데이터를 검색할 때 사용
display(df)
df.groupby('Team').filter(lambda x : len(x) >= 3)   # filter 안에는 boolean 조건이 존재해야 한다.
# len(x)는 grouped된 dataframe 개수

,Team,Rank,Year,Points
0,riders,1,2014,876
1,riders,2,2015,789
2,devils,2,2014,863
3,devils,3,2015,673
4,kings,3,2014,741
5,kings,4,2015,812
6,kings,1,2016,756
7,kings,1,2017,788
8,riders,2,2016,694
9,royals,4,2014,701


,Team,Rank,Year,Points
0,riders,1,2014,876
1,riders,2,2015,789
4,kings,3,2014,741
5,kings,4,2015,812
6,kings,1,2016,756
7,kings,1,2017,788
8,riders,2,2016,694
11,riders,2,2017,690


# Case study

In [28]:
import dateutil

df_phone = pd.read_csv('phone_data.csv')    # 시간과 데이터 종류가 정리된 통화량 데이터
df_phone['date'] = df_phone['date'].apply(dateutil.parser.parse, dayfirst = True)
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


In [29]:
df_phone.groupby('month')['duration'].sum()

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

In [30]:
df_phone[df_phone['item'] == 'call'].groupby('network')['duration'].sum()

network
Meteor        7200.0
Tesco        13828.0
Three        36464.0
Vodafone     14621.0
landline     18433.0
voicemail     1775.0
Name: duration, dtype: float64

In [31]:
# date 별 month, item 그룹 별 객체 수
df_phone.groupby(['month', 'item'])['date'].count()

month    item
2014-11  call    107
         data     29
         sms      94
2014-12  call     79
         data     30
         sms      48
2015-01  call     88
         data     31
         sms      86
2015-02  call     67
         data     31
         sms      39
2015-03  call     47
         data     29
         sms      25
Name: date, dtype: int64

In [32]:
# unstack
df_phone.groupby(['month', 'item'])['date'].count().unstack()

item,call,data,sms
month,,,
2014-11,107,29,94
2014-12,79,30,48
2015-01,88,31,86
2015-02,67,31,39
2015-03,47,29,25


In [33]:
df_phone.groupby('month', as_index = False).agg({'duration' : 'sum'})

,month,duration
0,2014-11,26639.441
1,2014-12,14641.870
2,2015-01,18223.299
3,2015-02,15522.299
4,2015-03,22750.441


In [34]:
df_phone.groupby('month', as_index = False).agg({'duration' : 'sum', 'network_type' : 'count', 'date' : 'first'})

,month,duration,network_type,date
0,2014-11,26639.441,230,2014-10-15 06:58:00
1,2014-12,14641.870,157,2014-11-13 06:58:00
2,2015-01,18223.299,205,2014-12-13 06:58:00
3,2015-02,15522.299,137,2015-01-13 06:58:00
4,2015-03,22750.441,101,2015-02-12 20:15:00


In [37]:
df_phone.groupby('month', as_index = False).agg({'duration' : [min, max, sum],  # find the min, max, sum
                                                'network_type' : 'count',      # find the number of network_type
                                                'date' : [min, 'first', 'nunique']}) # get the min, first date, unique

month duration                     network_type                date  \
                min      max        sum        count                 min   
0  2014-11      1.0   1940.0  26639.441          230 2014-10-15 06:58:00   
1  2014-12      1.0   2120.0  14641.870          157 2014-11-13 06:58:00   
2  2015-01      1.0   1859.0  18223.299          205 2014-12-13 06:58:00   
3  2015-02      1.0   1863.0  15522.299          137 2015-01-13 06:58:00   
4  2015-03      1.0  10528.0  22750.441          101 2015-02-12 20:15:00   

                               
                first nunique  
0 2014-10-15 06:58:00     210  
1 2014-11-13 06:58:00     147  
2 2014-12-13 06:58:00     172  
3 2015-01-13 06:58:00     125  
4 2015-02-12 20:15:00      93

In [38]:
grouped = df_phone.groupby('month', as_index = False).agg({'duration' : [min, max, np.mean]})

grouped.columns = grouped.columns.droplevel(level = 0)
grouped.rename(columns = {'min' : 'min_duration', 'max' : 'max_duration', 'mean' : 'mean_duration'})

,,min_duration,max_duration,mean_duration
0,2014-11,1.0,1940.0,115.823657
1,2014-12,1.0,2120.0,93.260318
2,2015-01,1.0,1859.0,88.894141
3,2015-02,1.0,1863.0,113.301453
4,2015-03,1.0,10528.0,225.251891


# Pivot table Crosstab
## Pivot table
- 우리가 excel에서 보던 그 것!
- Index 축은 groupby와 동일함
- Column에 추가로 labeling 값을 추가하여,
- Value에 numeric type 값을 aggregation 하는 형태

In [39]:
df_phone = pd.read_csv('phone_data.csv')
df_phone['date'] = df_phone['date'].apply(dateutil.parser.parse, dayfirst = True)
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


In [40]:
df_phone.pivot_table(['duration'],
                    index = [df_phone.month, df_phone.item],
                    columns = df_phone.network, aggfunc = 'sum', fill_value = 0)

duration                                                   \
network        Meteor Tesco  Three Vodafone      data landline special   
month   item                                                             
2014-11 call     1521  4045  12458     4316     0.000     2906       0   
        data        0     0      0        0   998.441        0       0   
        sms        10     3     25       55     0.000        0       1   
2014-12 call     2010  1819   6316     1302     0.000     1424       0   
        data        0     0      0        0  1032.870        0       0   
        sms        12     1     13       18     0.000        0       0   
2015-01 call     2207  2904   6445     3626     0.000     1603       0   
        data        0     0      0        0  1067.299        0       0   
        sms        10     3     33       40     0.000        0       0   
2015-02 call     1188  4087   6279     1864     0.000      730       0   
        data        0     0      0        0  1067.299        0       0   
        sms         1     2     11       23     0.000        0       2   
2015-03 call      274   973   4966     3513     0.000    11770       0   
        data        0     0      0        0   998.441        0       0   
        sms         0     4      5       13     0.000        0       0   

                              
network      voicemail world  
month   item                  
2014-11 call       301     0  
        data         0     0  
        sms          0     0  
2014-12 call       690     0  
        data         0     0  
        sms          0     4  
2015-01 call       285     0  
        data         0     0  
        sms          0     0  
2015-02 call       268     0  
        data         0     0  
        sms          0     0  
2015-03 call       231     0  
        data         0     0  
        sms          0     3

## Crosstab
- 특허 두 칼럼에 교차 빈도, 비율, 덧셈 등을 구할 때 사용
- Pivot table의 특수한 형태
- User-Item Rating Matrix 등을 만들 때 사용가능함

In [41]:
df_phone

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile
...,...,...,...,...,...,...,...
825,825,2015-03-13 00:38:00,1.000,sms,2015-03,world,world
826,826,2015-03-13 00:39:00,1.000,sms,2015-03,Vodafone,mobile
827,827,2015-03-13 06:58:00,34.429,data,2015-03,data,data
828,828,2015-03-14 00:13:00,1.000,sms,2015-03,world,world


In [42]:
pd.crosstab(index = df_phone.network, 
            columns = df_phone.network_type, 
            values = df_phone.duration, 
            aggfunc = 'first').fillna(0)

network_type,data,landline,mobile,special,voicemail,world
network,,,,,,
Meteor,0.000,0.0,23.0,0.0,0.0,0.0
Tesco,0.000,0.0,4.0,0.0,0.0,0.0
Three,0.000,0.0,602.0,0.0,0.0,0.0
Vodafone,0.000,0.0,13.0,0.0,0.0,0.0
data,34.429,0.0,0.0,0.0,0.0,0.0
landline,0.000,6.0,0.0,0.0,0.0,0.0
special,0.000,0.0,0.0,1.0,0.0,0.0
voicemail,0.000,0.0,0.0,0.0,19.0,0.0
world,0.000,0.0,0.0,0.0,0.0,1.0


In [46]:
df_phone.pivot_table(['duration'],
                    index = df_phone.network, 
                    columns = df_phone.network_type,
                    aggfunc = 'first',
                    fill_value = 0)

duration                                        
network_type     data landline mobile special voicemail world
network                                                      
Meteor          0.000        0     23       0         0     0
Tesco           0.000        0      4       0         0     0
Three           0.000        0    602       0         0     0
Vodafone        0.000        0     13       0         0     0
data           34.429        0      0       0         0     0
landline        0.000        6      0       0         0     0
special         0.000        0      0       1         0     0
voicemail       0.000        0      0       0        19     0
world           0.000        0      0       0         0     1

# merge & concat
- SQL에서 많이 사용하는 Merge와 같은 기능
- 두 개의 데이터를 하나로 합침

In [53]:
df_a = pd.DataFrame({'subject_id' : [1, 2, 3, 4, 5, 7], 'test_score' : [51, 15, 15, 61, 16, 14]}, columns = ['subject_id', 'test_score'])
df_a

,subject_id,test_score
0,1,51
1,2,15
2,3,15
3,4,61
4,5,16
5,7,14


In [56]:
df_b = pd.DataFrame({'subject_id' : [4, 5, 6, 7, 8], 'first_name' : ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 'last_name' : ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}, columns = ['subject_id', 'first_name', 'last_name'])
df_b

,subject_id,first_name,last_name
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


In [58]:
# subject_id 기준 merge
pd.merge(df_a, df_b, on = 'subject_id')

,subject_id,test_score,first_name,last_name
0,4,61,Billy,Bonder
1,5,16,Brian,Black
2,7,14,Bryce,Brice


In [57]:
# 두 dataframe의 columns 이름이 다를 때
pd.merge(df_a, df_b, left_on = 'subject_id', right_on = 'subject_id')

,subject_id,test_score,first_name,last_name
0,4,61,Billy,Bonder
1,5,16,Brian,Black
2,7,14,Bryce,Brice


In [59]:
# left join
pd.merge(df_a, df_b, on = 'subject_id', how = 'left')

,subject_id,test_score,first_name,last_name
0,1,51,NaN,NaN
1,2,15,NaN,NaN
2,3,15,NaN,NaN
3,4,61,Billy,Bonder
4,5,16,Brian,Black
5,7,14,Bryce,Brice


In [60]:
# right join
pd.merge(df_a, df_b, on = 'subject_id', how = 'right')

,subject_id,test_score,first_name,last_name
0,4,61.0,Billy,Bonder
1,5,16.0,Brian,Black
2,6,NaN,Bran,Balwner
3,7,14.0,Bryce,Brice
4,8,NaN,Betty,Btisan
